In [7]:
# ==============================================================================
# 02_Model.ipynb - Entraînement du Modèle de Prédiction (Régression Logistique)
# ==============================================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score, classification_report
import joblib # Pour la sauvegarde du modèle

# --- 1. Chargement et Préparation des Données ---

print("--- 1. Chargement des données enrichies ---")

df = pd.read_csv('../data/processed/can_processed_data_with_elo.csv')
df['date'] = pd.to_datetime(df['date'])

# **********************************************
# CORRECTION DES NaN (Résout : ValueError: Input X contains NaN)

print("Vérification des données manquantes avant modélisation...")

# Colonnes principales qui pourraient contenir des NaN après la fusion: points FIFA et ELO
df.dropna(subset=['home_points', 'away_points', 'home_elo', 'away_elo'], inplace=True)

print(f"Lignes avec NaN dans les features supprimées. Nouvelle taille: {df.shape}")
# **********************************************

print(f"Dataset chargé. Taille: {df.shape}")
# ... (le reste du code)
# --- 2. Création des Variables Cibles (Y) et des Features (X) ---

# 2.1. Création de la variable cible (Y): Résultat du match
# 1 = Victoire Domicile (Home Win)
# 0 = Match Nul (Draw)
# -1 = Défaite Domicile (Home Loss / Away Win)

def get_match_result(row):
    if row['home_score'] > row['away_score']:
        return 1.0 
    elif row['home_score'] < row['away_score']:
        return -1.0 
    else:
        return 0.0 

df['result'] = df.apply(get_match_result, axis=1)

# 2.2. Création des variables d'entrée (X)
print("Création des features (différence ELO, points FIFA)...")

# Features principales : la différence de force
df['elo_diff'] = df['home_elo'] - df['away_elo']
df['points_diff'] = df['home_points'] - df['away_points']

# Feature Avantage du terrain (constante utilisée dans le script ELO)
HOME_FIELD_ADVANTAGE_FEATURE = 100 
df['home_adv'] = HOME_FIELD_ADVANTAGE_FEATURE

# Sélection des features
X = df[['elo_diff', 'points_diff', 'home_adv']]
Y = df['result']

# --- 3. Séparation des Données d'Entraînement et de Test ---

# Séparation temporelle (Entraînement sur les matchs avant 2022, Test sur les matchs récents)
split_date = pd.to_datetime('2022-01-01') 

df_train = df[df['date'] < split_date]
df_test = df[df['date'] >= split_date]

X_train = df_train[['elo_diff', 'points_diff', 'home_adv']]
Y_train = df_train['result']

X_test = df_test[['elo_diff', 'points_diff', 'home_adv']]
Y_test = df_test['result']

print(f"Taille du set d'entraînement (avant {split_date.strftime('%Y-%m-%d')}): {X_train.shape[0]}")
print(f"Taille du set de test (après {split_date.strftime('%Y-%m-%d')}): {X_test.shape[0]}")


# --- 4. Entraînement du Modèle ---

print("\n--- 4. Entraînement du Modèle de Régression Logistique ---")

# La Régression Logistique est utilisée pour prédire la probabilité (la Régression Logistique est un classifieur).
# 
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42, max_iter=200)
model.fit(X_train, Y_train)

print("Modèle entraîné avec succès.")


# --- 5. Évaluation du Modèle ---

print("\n--- 5. Évaluation des Performances ---")

# Prédictions
Y_pred = model.predict(X_test)
Y_proba = model.predict_proba(X_test)

# Log Loss (Mesure de la confiance des probabilités, clé pour la prédiction de paris)
logloss = log_loss(Y_test, Y_proba, labels=[-1.0, 0.0, 1.0])
print(f"Log Loss sur le set de test: {logloss:.4f}")

# Précision (Accuracy)
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Précision (Accuracy) sur le set de test: {accuracy*100:.2f}%")

# Rapport détaillé
print("\nRapport de Classification:")
print(classification_report(Y_test, Y_pred, target_names=['Away Win', 'Draw', 'Home Win']))


# --- 6. Sauvegarde du Modèle ---

# Sauvegarder le modèle entraîné pour l'utiliser plus tard
model_filename = '../models/logistic_regression_model.joblib'
joblib.dump(model, model_filename)

print(f"\n✅ Modèle sauvegardé dans {model_filename}")

print("\nLe modèle est prêt pour la phase de prédiction !")

--- 1. Chargement des données enrichies ---
Vérification des données manquantes avant modélisation...
Lignes avec NaN dans les features supprimées. Nouvelle taille: (5700, 13)
Dataset chargé. Taille: (5700, 13)
Création des features (différence ELO, points FIFA)...
Taille du set d'entraînement (avant 2022-01-01): 4970
Taille du set de test (après 2022-01-01): 730

--- 4. Entraînement du Modèle de Régression Logistique ---
Modèle entraîné avec succès.

--- 5. Évaluation des Performances ---
Log Loss sur le set de test: 0.9152
Précision (Accuracy) sur le set de test: 58.36%

Rapport de Classification:
              precision    recall  f1-score   support

    Away Win       0.61      0.57      0.59       197
        Draw       0.33      0.01      0.02       190
    Home Win       0.58      0.91      0.71       343

    accuracy                           0.58       730
   macro avg       0.51      0.50      0.44       730
weighted avg       0.52      0.58      0.50       730


✅ Modèle sa